In [110]:
from numpy import *
import matplotlib.pyplot as plt

---

# Config

In [111]:
L = 10 # Size of the grid
D1 = 1e4 # Diffusion coefficient [s-1]
F = linspace(5e-6,1e4,1000,endpoint=True) # Flux [s-1]

simulation_time = 10 # [s]

---

# Class definition

In [112]:
class Layer:
    def __init__(self, L):
        self.grid = []
        for i in range(L):
            self.grid.append([])
            for j in range(L):
                self.grid[i].append(None)
        self.islands = []
    
    def elements(self):
        monomers = []
        islands = []
        for row in self.grid:
            for monomer in row:
                if monomer is not None:
                    if monomer.island is None:
                        monomers.append(monomer)
                    else:
                        if monomer.island not in islands:
                            islands.append(monomer.island)
        return monomers, islands

    def monomers(self):
        return self.elements()[0]

    def islands(self):
        return self.elements()[1]

    def heightmap(self):
        heightmap = zeros((L,L))
        for x in range(L):
            for y in range(L):
                if self.grid[x][y] is not None:
                    heightmap[x][y] = 1
        return heightmap

In [113]:
class Monomer: 
    def __init__(self, layer, x=None, y=None):
        self.layer = layer
        self.island = None
        
        # Place the monomer randomly on the layer
        while True:
            cpt = 0
            if x is None:
                self.x = random.randint(0, L)
            else:
                self.x = x
            if y is None:
                self.y = random.randint(0, L)
            else: self.y = y
            
            # Add the monomer to the layer
            if layer.grid[self.x ][self.y] is None:
                layer.grid[self.x][self.y] = self
                break

            if cpt > 1000: print("More than 1000 iteration to find a free spot... (it's recommended to stop the program)")
    
    def neighbors(self):
        neighbors = []
        for x in range(self.x - 1, self.x + 2):
            for y in range(self.y - 1, self.y + 2):
                if x >= 0 and x < L and y >= 0 and y < L and (x != self.x or y != self.y):
                    if self.layer.grid[x][y] is not None:
                        neighbors.append(self.layer.grid[x][y])
        return neighbors

    def move(self):
        if self.island is not None: return

        # Choose a random direction
        self.x += random.randint(-1, 2)
        self.y += random.randint(-1, 2)

        # If the monomer is out of the layer, it is teleportated to the other side
        if self.x < 0:
            self.x = 0
        if self.x >= L:
            self.x = L - 1
        if self.y < 0:
            self.y = 0
        if self.y >= L:
            self.y = L - 1

        if len(neighbors := self.neighbors()) == 0:
            for monomer in neighbors:
                if monomer.island is not None:
                    monomer.island.add_monomer(self)
                    break
            self.layer.grid[self.x][self.y] = self

In [114]:
class Island:
    def __init__(self, layer, monomers):
        self.layer = layer
        self.monomers = monomers
    
    def add_monomer(self, monomer):
        self.occupied_sites.append((monomer.x, monomer.y))
        monomer.island = self
                

---
# Simulation

In [117]:
layer = Layer(L)
evolution = []

def evolve():
    monomer = Monomer(layer)

    for monomer in layer.monomers():
        monomer.move()

        if len(neighbors := monomer.neighbors()) > 0:
            island = Island(layer, neighbors + [monomer])

    return layer.heightmap()

from LRFutils import progress
a = progress.Bar(max=1000)
for i, simulation_step in enumerate(linspace(0,simulation_time,1000)):
    evolution.append(evolve())
    a(i)
a(1000)

import analyze
analyze.generate_animation(evolution, save_as = None, plot=False, verbose = False)

[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None]


KeyboardInterrupt: 